In [56]:
import polars as pl
from pathlib import Path

data_dir = Path("../data")

transactions = pl.scan_parquet(data_dir / "transactions_train_3.parquet")
lines = pl.scan_parquet(data_dir / "transaction_lines_train_3.parquet")


In [57]:
joined = transactions.join(
  lines,
  left_on="id",
  right_on="transaction_id",
  how="left",
  suffix="_line",
)

camera_frauds = (
  joined
  .filter(pl.col("label") != "UNKNOWN")
  .filter(pl.col("was_voided") & (pl.col("sales_price") == 0))
  .select(pl.col("id").alias("transaction_id"), "label", "damage", "camera_product_similar", "camera_certainty")
 )

# Checks 

- bei allen handelt es sich um FRAUD 
- camera_product_similar ist False

In [58]:
(camera_frauds
 .with_columns((pl.col("label") == "FRAUD").alias("is_fraud"))
 .select(
    pl.col("is_fraud").all(),
    pl.col("camera_product_similar").not_().all()
)).collect()

is_fraud,camera_product_similar
bool,bool
true,true


In [59]:
camera_frauds.describe()

statistic,transaction_id,label,damage,camera_product_similar,camera_certainty
str,str,str,f64,f64,f64
"""count""","""500""","""500""",500.0,500.0,500.0
"""null_count""","""0""","""0""",0.0,0.0,0.0
"""mean""",null,null,14.94988,0.0,0.955847
"""std""",null,null,14.67362,null,0.062179
"""min""","""00b6104a-29dc-491b-8687-129ff4…","""FRAUD""",0.19,0.0,0.639371
"""25%""",null,null,4.36,null,0.923147
"""50%""",null,null,11.12,null,0.992971
"""75%""",null,null,20.45,null,1.0
"""max""","""ff6d257e-9a6e-410c-9a71-080fc6…","""FRAUD""",83.9,0.0,1.0


# Summary

In [61]:
n_lines = camera_frauds.select(pl.len()).collect()[0,0]
n_transactions = camera_frauds.select("transaction_id").unique().select(pl.len()).collect()[0, 0]
n_frauds = transactions.filter(pl.col("label") == "FRAUD").select(pl.len()).collect()[0, 0]

total_damage_non_scanned = (camera_frauds.group_by("transaction_id")
  .agg([
    pl.col("label").first().alias("label"),
    pl.col("damage").first().alias("damage"),
  ])
).select(pl.col("damage").sum().alias("total_damage")).collect()[0,0]

total_damage = transactions.select(pl.col("damage").sum().alias("total_damage")).collect()[0,0]

In [63]:
def print_aligned(text, value):
    if isinstance(value, float):
      print(f"{text: <50} {value:>10.2f}")
    else:
      print(f"{text: <50} {value:>10}")

# Anzahl Transaktionen mit FRAUD, Anzahl nicht gescannte Lines, Anzahl Transaktion mit nicht gescannten Lines
print_aligned("Anzahl nicht gescannte Lines:", n_lines)
print_aligned("Anzahl Transaktion mit nicht gescannten Lines:", n_transactions)
print_aligned("Anzahl Transaktionen mit FRAUD:", n_frauds)
print_aligned("Anteil:", n_frauds / n_transactions)

# Schaden der nicht gescannten Lines, Schaden aller Transaktionen
print_aligned("Schaden der nicht gescannten Lines:", total_damage_non_scanned)
print_aligned("Schaden aller Transaktionen:", total_damage)
# Anteil Schaden nicht gescannter Lines am Gesamtschaden
print_aligned("Anteil Schaden nicht gescannter Lines:", total_damage_non_scanned / total_damage * 100)


Anzahl nicht gescannte Lines:                             500
Anzahl Transaktion mit nicht gescannten Lines:            377
Anzahl Transaktionen mit FRAUD:                          4656
Anteil:                                                 12.35
Schaden der nicht gescannten Lines:                   5088.38
Schaden aller Transaktionen:                         35172.12
Anteil Schaden nicht gescannter Lines:                  14.47
